In [3]:
import numpy as np
import pandas as pd
from collections import defaultdict

In [4]:
df = pd.read_excel("./All.xlsx",engine='openpyxl')
df[0:5]

,User,Place,Rating
0,江宛俶,B.B.ART,5
1,Hera Liu,B.B.ART,4
2,許詠貽,B.B.ART,5
3,Yeh JC,B.B.ART,5
4,黃燕伶,B.B.ART,4


In [5]:
# 輸入景點 推薦其他人去的景點

In [25]:
Name = '南鯤鯓代天府'
UserList = df.loc[df["Place"]==Name,"User"].values
UserOne = UserList[0]
SetList = []
user_similarity = [] 

for other_user in UserList[0:50]: 
    if len(df.loc[df["User"]==other_user,"Place"].values) < 5 :continue
    if UserOne == other_user:continue
    if {UserOne,other_user} not in SetList:
        SetList.append({UserOne,other_user})
    else:
        continue
    sim = find_common_places(UserOne,other_user) # 兩者相似度的分數 
    user_similarity.append([other_user,sim]) # 將使用者 & 兩者相似度的分數 變 List  
        
user_similarity = np.array(user_similarity) # 將使用者 & 兩者相似度的分數 轉成 np
sorted_index = np.argsort(user_similarity, axis=0)[:,1][::-1][:10] # 依照排序
top10_similar_user = user_similarity[:,0][sorted_index] # 將排序帶入使用者

wordCountList = []
wordCountDict = dict()
for inter in top10_similar_user:
    movies = df.loc[df.User==inter,["Place","Rating"]].values.tolist()
    for re,sor in movies:
        if sor == 5 and re != Name:
            if re in wordCountDict:
                wordCountDict[re] += 1
            else:
                wordCountDict[re] = 1
    wordCountList = [(w,wordCountDict[w]) for w in wordCountDict]
    wordCountList.sort(key=lambda x:x[1],reverse=True)


In [28]:
for recom,score in wordCountList[0:5]:
    print(recom)

二寮日出
國立臺灣歷史博物館
奇美食品幸福工廠
原台南州廳-台灣文學館
延平郡王祠


In [ ]:
top10_similar_user

In [8]:
def find_common_places(user1,user2):
    u1 = df[df["User"]==user1] # 條件 = user1 的 DF
    u2 = df[df["User"]==user2] # 條件 = user2 的 DF   
    s1 = set((df.loc[df["User"]==user1,"Place"].values)) #取出 user1 所有去過的景點
    s2 = set((df.loc[df["User"]==user2,"Place"].values)) #取出 user2 所有去過的景點
    place_id = s1.intersection(s2) # 取交集
    vec1 = u1[u1.Place.isin(place_id)].sort_values(by="Place")["Rating"].values # 找出相對應的景點評分
    vec2 = u2[u2.Place.isin(place_id)].sort_values(by="Place")["Rating"].values # 找出相對應的景點評分
    vec1 = np.mat(vec1)
    vec2 = np.mat(vec2)
    num = float(vec1 * vec2.T)
    denom = np.linalg.norm(vec1) * np.linalg.norm(vec2)
    cos = num / denom
    sim = 0.5 + 0.5 * cos
    return sim 
    